<a href="https://colab.research.google.com/github/btcnhung1299/tf-practice/blob/master/TF_Traffic_Sign_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

## Data Gathering

Download data from a given URL.

In [2]:
!wget https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
!unzip -q traffic-signs-data.zip

--2020-09-03 15:05:49--  https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip
Resolving d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)... 13.226.47.146, 13.226.47.136, 13.226.47.88, ...
Connecting to d17h27t6h515a5.cloudfront.net (d17h27t6h515a5.cloudfront.net)|13.226.47.146|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  59.8MB/s    in 2.0s    

2020-09-03 15:05:51 (59.8 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]



Read data from pickle files.

In [3]:
import pickle

train_data = pickle.load(open("train.p", "rb"))
val_data = pickle.load(open("valid.p", "rb"))
test_data = pickle.load(open("test.p", "rb"))

In [4]:
get_features_labels = lambda x : (x["features"], x["labels"])
X_train, y_train = get_features_labels(train_data)
X_val, y_val = get_features_labels(val_data)
X_test, y_test = get_features_labels(test_data)

Define the input shape for modeling.

In [5]:
num_train_samples, *input_shape = X_train.shape
num_classes = 43
print("Input shape:", input_shape)

Input shape: [32, 32, 3]


Since the pixel lies between 0-255, need **scaling** -> would add a rescaling layer later in the model.

In [6]:
print("Pixel range:", np.min(X_train), np.max(X_train))

Pixel range: 0 255


## Data Shuffling

In [7]:
from sklearn.utils import shuffle

X_train, y_train = shuffle(X_train, y_train)
X_val, y_val = shuffle(X_val, y_val)
X_test, y_test = shuffle(X_test, y_test)

## Data Augmentation

In [8]:
BATCH_SIZE = 128

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

augmentator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2, 
    horizontal_flip=True
)

ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
ds_train_aug = augmentator.flow(X_train, y_train, batch_size=BATCH_SIZE)

## Model Architecture

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [22]:
model = Sequential()

model.add(Rescaling(scale=1./255, input_shape=input_shape))

model.add(Conv2D(filters=32, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (32 x 32 x 32)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (16 x 16 x 32)

model.add(Conv2D(filters=64, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(2, 2), padding="same", activation="relu")) # Input shape: (16 x 16 x 64)
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))                                            # Input shape: (8 x 8 x 64)

model.add(Flatten())                                                                 # Input shape: (4096)
model.add(Dense(512, activation="relu"))                                             # Input shape: (512)
model.add(Dense(num_classes, activation="softmax"))                                  # Input shape: (num_classes)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_1 (Rescaling)      (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_4 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_5 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 16, 16, 64)       

In [23]:
rmsprop_opt = tf.optimizers.RMSprop()
model.compile(loss="sparse_categorical_crossentropy", optimizer=rmsprop_opt, metrics=["acc"])

## Training

In [24]:
history = model.fit(ds_train, epochs=20, validation_data=(X_val, y_val))

Epoch 1/50
272/272 [==============================] - 3s 10ms/step - loss: 0.9727 - acc: 0.8032 - val_loss: 2.7460 - val_acc: 0.4624
Epoch 2/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0676 - acc: 0.9822 - val_loss: 0.6109 - val_acc: 0.8655
Epoch 3/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0349 - acc: 0.9912 - val_loss: 0.3341 - val_acc: 0.9388
Epoch 4/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0287 - acc: 0.9936 - val_loss: 0.4005 - val_acc: 0.9447
Epoch 5/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0187 - acc: 0.9959 - val_loss: 0.5240 - val_acc: 0.9313
Epoch 6/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0131 - acc: 0.9972 - val_loss: 0.4826 - val_acc: 0.9501
Epoch 7/50
272/272 [==============================] - 3s 10ms/step - loss: 0.0109 - acc: 0.9977 - val_loss: 0.4271 - val_acc: 0.9512
Epoch 8/50
272/272 [==============================] - 3s 10ms/step - 

In [25]:
model.save("traffic_sign.h5")

## Evaluation

In [26]:
saved_model = tf.keras.models.load_model("traffic_sign.h5")

In [28]:
saved_model.evaluate(X_test, y_test)

395/395 [==============================] - 1s 2ms/step - loss: 0.7098 - acc: 0.0045


[0.7098062038421631, 0.004513063933700323]